For Aassigment 5 Machine Translation: IBM Model 1

In [2]:
# write code to collect statistics about word frequencies in the two languages. 
# Print the 10 most frequent words in each language.
# using a Counter
from collections import Counter
import re

source="./dat410_europarl/europarl-v7.fr-en.lc.en"
target="./dat410_europarl/europarl-v7.fr-en.lc.fr"

def get_words(filename):
    with open(filename, 'r') as f:
        text = f.read()
    words = re.findall(r'\w+', text.lower())
    return words

def get_10_most_frequent_words():
    english_words = get_words(source)
    french_words = get_words(target)
    english_word_counts = Counter(english_words)
    french_word_counts = Counter(french_words)
    print('English:', english_word_counts.most_common(10))
    print('French:', french_word_counts.most_common(10))

get_10_most_frequent_words()

# print the probabitly of "zebra" in the two languages
english_words = get_words(source)
english_word_counts = Counter(english_words)
print("English:%f%%" % (english_word_counts['speaker'] / len(english_words) * 100)) 
print("English:%f%%" % (english_word_counts['zebra'] / len(english_words) * 100))

English: [('the', 19627), ('of', 9534), ('to', 8992), ('and', 7214), ('in', 6197), ('is', 4453), ('that', 4421), ('a', 4388), ('we', 3341), ('this', 3332)]
French: [('apos', 16729), ('de', 14528), ('la', 9746), ('et', 6620), ('l', 6536), ('le', 6177), ('à', 5588), ('les', 5587), ('des', 5232), ('que', 4797)]
English:0.004571%
English:0.000000%


In [3]:
# bigram language model

from math import log,exp

class BigramLanguageModel:
    def __init__(self):
        pass

    def train(self, words):
        self.bigrams = Counter(zip(words, words[1:]))
        self.unigrams = Counter(words)
        self.total = len(words)
        
    def probability_w(self, word1, word2):
        if self.unigrams[word1] == 0 \
            or self.unigrams[word2] ==0 \
            or self.bigrams[(word1,word2)] == 0:
            # if word non exist
            return 1.0 / (self.total + 1)
        return self.bigrams[(word1,word2)]/self.unigrams[word1]
    
    def predict(self, words: list):
        # length of words should be at least 1
        if len(words) == 0: return 1
        Ps=1.0 # the probability_w (words[0]|<start>) is 1
        for i in range(1,len(words)):
            pt = self.probability_w(words[i-1],words[i])
            Ps = Ps* pt
            print("P(%s|%s) is %.15f \t P(S): %.15f" %(words[i],words[i-1], pt, Ps))
            # print("\t (%s, %s):%d, (%s): %d" %(words[i-1],words[i],
            #                                    self.bigrams[(words[i-1],words[i])], 
            #                                    words[i-1],
            #                                    self.unigrams[words[i-1]]))
        return Ps
    
    def predict_log(self, words: list):
        # length of words should be at least 1
        if len(words) == 0: return 1
        Ps=0.0 # the probability_w (words[0]|<start>) is 1
        for i in range(1,len(words)):
            pt = log(self.probability_w(words[i-1],words[i]))
            Ps = Ps + pt
            # print("P(%s|%s) is %.15f \t P(S): %.15f" %(words[i],words[i-1], pt, Ps))
            # print("\t (%s, %s):%d, (%s): %d" %(words[i-1],words[i],
            #                                    self.bigrams[(words[i-1],words[i])], 
            #                                    words[i-1],
            #                                    self.unigrams[words[i-1]]))
        return exp(Ps)

En_bigramM = BigramLanguageModel()
En_bigramM.train(english_words)



# print(english_words[:100])

# debug
# En_bigamM.probability_w('please','open') 
# En_bigamM.bigrams.most_common(10)
# 

# case 1 normal short sentence "the house rose", from dataset
print("----- Normal case")
sentence = "the house rose"
print("P('%s') is %.15f, lP: %.15f " % (sentence, 
                En_bigramM.predict(sentence.split(' ')),
                En_bigramM.predict_log(sentence.split(' '))))
print("-----")
sentence = "it says that this should be done"
print("P('%s') is %.15f, lP: %.15f " % (sentence, 
                En_bigramM.predict(sentence.split(' ')),
                En_bigramM.predict_log(sentence.split(' '))))

# case 2 words non exist
print("----- Non exist")
sentence = "the house rose NNN"
print("P('%s') is %.15f, lP: %.15f " % (sentence, 
                En_bigramM.predict(sentence.split(' ')),
                En_bigramM.predict_log(sentence.split(' '))))

# case 3 words super length
print("----- over length")
sentence = "you will be aware from the press and television that there have been a number of bomb explosions and killings in sri lanka"
print("len(%d) ,P('%s ...') is %.15f, lP: %.15f " % (len(sentence),
                sentence[:10],
                En_bigramM.predict(sentence.split(' ')),
                En_bigramM.predict_log(sentence.split(' '))))

----- Normal case
P(house|the) is 0.002139909308605 	 P(S): 0.002139909308605
P(rose|house) is 0.006329113924051 	 P(S): 0.000013543729801
P('the house rose') is 0.000013543729801, lP: 0.000013543729801 
-----
P(says|it) is 0.001999200319872 	 P(S): 0.001999200319872
P(that|says) is 0.529411764705882 	 P(S): 0.001058400169344
P(this|that) is 0.046821985976024 	 P(S): 0.000049556397886
P(should|this) is 0.003901560624250 	 P(S): 0.000000193347291
P(be|should) is 0.295402298850575 	 P(S): 0.000000057115234
P(done|be) is 0.016312594840668 	 P(S): 0.000000000931698
P('it says that this should be done') is 0.000000000931698, lP: 0.000000000931698 
----- Non exist
P(house|the) is 0.002139909308605 	 P(S): 0.002139909308605
P(rose|house) is 0.006329113924051 	 P(S): 0.000013543729801
P(NNN|rose) is 0.000003808972415 	 P(S): 0.000000000051588
P('the house rose NNN') is 0.000000000051588, lP: 0.000000000051588 
----- over length
P(will|you) is 0.052256532066508 	 P(S): 0.052256532066508
P(be|wi

In [27]:
# translate model IBM1

class IBM1():
    def __init__(self):
        self.t = {}
    
    def train(self, source_sentences, target_sentences, n_iter=5):
        # generate and initialize t(e|f), c(e -> f), c(e) uniformly
        c_e_f={}
        c_e={}
        print("Initializing t")
        f_words = set()
        e_words = set()
        for k in range(len(source_sentences)):
            f_sentences = source_sentences[k]
            e_sentences = target_sentences[k]
            f_words.add(word for word in re.findall(r'\w+', f_sentences))
            e_words.add(word for word in re.findall(r'\w+', f_sentences))
            print(f_words)
        for e in e_words:
            c_e[e] = 0.0
        for f in f_words:
            for e in e_words:
                self.t[(e,f)] = 1.0
                c_e_f[(e,f)] = 0.0    
                if f =='la':
                    print("t(%s|%s) = %.15f" % (e,f, self.t[(e,f)]))
                    
        # start iteration
        print("Start iteration")
        def single_iteration():
            # set c(e -> f) and c(e) to 0
            for key in c_e.keys():
                c_e[key] = 0.0
            for key in c_e_f.keys():
                c_e_f[key] = 0.0
            # update c(e -> f) and c(e)
            for k in range(len(source_sentences)):
                f_sentences = source_sentences[k]
                e_sentences = target_sentences[k]
                
                f_words = re.findall(r'\w+', f_sentences)
                e_words = re.findall(r'\w+', e_sentences)
                
                for i in range(len(f_words)):
                    f = f_words[i]
                    esum = sum([self.t[(ej,f)] for ej in e_words])
                    for j in range(len(e_words)):
                        e = e_words[j]
                        theta = self.t[(e,f)]/esum
                        c_e_f[(e,f)] += theta
                        c_e[e] += theta
            # update t(e|f)
            for k in range(len(source_sentences)):
                f_sentences = source_sentences[k]
                e_sentences = target_sentences[k]
                
                f_words = re.findall(r'\w+', f_sentences)
                e_words = re.findall(r'\w+', e_sentences)
                
                for f in f_words:
                    for e in e_words:
                        self.t[(e,f)] = c_e_f[(e,f)]/c_e[e]
                        
        for i in range(n_iter):
            single_iteration()
            print("iteration %d" % i)         
        
                            
        

source_sentences = open(source, 'r').readlines()
target_sentences = open(target, 'r').readlines()
print("len(source_sentences): %d, len(target_sentences): %d" % (len(source_sentences), len(target_sentences)))

ibm1 = IBM1()
# how many iteration we should run?
ibm1.train(source_sentences, target_sentences, n_iter=1)
print(ibm1.t[('the','le')])

len(source_sentences): 10000, len(target_sentences): 10000
Initializing t
len(f_words): 10000, len(e_words): 10000


KeyboardInterrupt: 